# Manning Chapter 2: Qubits

In [6]:
from __future__ import annotations

import numpy as np
from IPython.display import display

from abc import ABCMeta, abstractmethod
from contextlib import contextmanager

## Basic qubits

### ket

`ket`表示的是**state**（状态），常用的为：

$$\left| 0 \right> = \begin{bmatrix}1\\0\end{bmatrix}$$
$$\left| 1 \right> = \begin{bmatrix}0\\1\end{bmatrix}$$
$$\left| + \right> = (\left| 0 \right> + \left| 1 \right>)/\sqrt{2}$$
$$\left| - \right> = (\left| 0 \right> - \left| 1 \right>)/\sqrt{2}$$

In [3]:
ket0 = np.array([[1], [0]])
ket1 = np.array([[0], [1]])
ket_plus = (ket0 + ket1) / np.sqrt(2)
ket_minus = (ket0 - ket1) / np.sqrt(2)

[print(x) for x in [ket0, ket1, ket_plus, ket_minus]]

[[1]
 [0]]
[[0]
 [1]]
[[0.70710678]
 [0.70710678]]
[[ 0.70710678]
 [-0.70710678]]


[None, None, None, None]

### bra

`bra` 表示测量，并且满足

$$P(测量|状态) = |\left<测量 \middle| 状态\right>|^2$$

其中，$\left<x\middle|y\right>$表示内积。

### 泡利矩阵



$$\sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$$

满足

$$\sigma_x \left| 0 \right> = \left| 1 \right>$$
$$\sigma_x \left| 1 \right> = \left| 0 \right>$$

### Hadamard算子

$$H = \frac{1}{\sqrt{2}}\begin{pmatrix}1 & 1\\1 & -1\end{pmatrix}$$

In [4]:
hadamard_operator = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

**Hadamard算子** $H$满足：

$$H \left| 0 \right> = \left| + \right>$$
$$H \left| 1 \right> = \left| - \right>$$

In [5]:
display(hadamard_operator @ ket0 == ket_plus)
display(hadamard_operator @ ket1 == ket_minus)

array([[ True],
       [ True]])

array([[ True],
       [ True]])

## 产生随机数的例子

In [17]:
class Qubit(metaclass=ABCMeta):
    """量子比特抽象类
    """
    @abstractmethod
    def h(self):
        """Hadamard算子
        """
    
    @abstractmethod
    def measure(self) -> bool:
        """测量
        """
    
    @abstractmethod
    def reset(self):
        """重置
        """

class QuantumDevice(metaclass=ABCMeta):
    """量子设备
    """
    @abstractmethod
    def allocate_qubit(self) -> Qubit:
        """获取量子比特
        """
        pass
    
    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit):
        """释放量子比特
        """
        pass
    
    @contextmanager
    def using_qubit(self):
        """生成量子比特
        """
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)

In [18]:
def qrng(device: QuantumDevice) -> bool:
    """产生随机数
    """
    with device.using_qubit() as q:
        q.h()
        return q.measure()

In [19]:
class SimulateQubit(Qubit):
    """量子比特模拟器
    """
    def __init__(self):
        self.reset()
        
    def h(self):
        """Hadamard算子
        """
        self.state = hadamard_operator @ self.state
    
    def measure(self) -> bool:
        """测量
        """
        pr0 = np.abs(self.state[0, 0]) ** 2
        sample = np.random.random() <= pr0
        return bool(0 if sample else 1)
    
    def reset(self):
        """重置
        """
        self.state = ket0.copy()

In [20]:
class SingleQubitSimulator(QuantumDevice):
    """单量子比特模拟器
    """
    def allocate_qubit(self) -> Qubit:
        return SimulateQubit()
    
    def deallocate_qubit(self, qubit: Qubit):
        pass

In [22]:
## 打印结果
qsim = SingleQubitSimulator()
for i in range(10):
    sample = qrng(qsim)
    print(f"{sample}")

False
False
True
False
False
True
False
True
True
True
